In [1]:
!pip install gensim

You should consider upgrading via the '/Users/max/.pyenv/versions/3.7.11/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
!pip install nltk

You should consider upgrading via the '/Users/max/.pyenv/versions/3.7.11/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:
!pip install texthero

In [5]:
!pip install sentence-transformers

You should consider upgrading via the '/Users/max/.pyenv/versions/3.7.11/bin/python3.7 -m pip install --upgrade pip' command.


In [32]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import tensorflow_hub as hub
from nltk.tokenize import word_tokenize
from scipy import spatial
import nltk
import texthero as hero
from sentence_transformers import SentenceTransformer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/max/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

here we generate the dataframe, which keyword named as title and GT recommendation named as content.

We are using here just the NYT articles, as buy using articles from just one publisher we hope to get a better more consistant matching and less confusion, which is important, when building the pipeline

In [33]:
df = pd.read_csv('../comparison_dataset/articles1.csv')
df = df[df['publication'] == 'New York Times']
df = df.drop(columns = ['id', 'publication', 'author', 'date', 'year', 'month', 'url', 'Unnamed: 0'])
df

,title,content
0,House Republicans Fret About Winning Their Hea...,WASHINGTON — Congressional Republicans have...
1,Rift Between Officers and Residents as Killing...,"After the bullet shells get counted, the blood..."
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...","When Walt Disney’s “Bambi” opened in 1942, cri..."
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...","Death may be the great equalizer, but it isn’t..."
4,Kim Jong-un Says North Korea Is Preparing to T...,"SEOUL, South Korea — North Korea’s leader, ..."
...,...,...
7798,U.N. Relief Official Calls Crisis in Aleppo th...,The top aid official at the United Nations gav...
7799,Federal Judge Curbs Enforcement of North Carol...,A federal judge on Friday curbed the enforceme...
7800,Mexicans Accuse President of ‘Historic Error’ ...,MEXICO CITY — If President Enrique Peña Nie...
7801,"U.S. Presses for Truce in Syria, With Its Larg...","HANGZHOU, China — The image of a Syrian ..."


now we try to find embedding methods, which in an ideal matching, or we could also just try to minimize distance between the embedded title and content

In [ ]:
df['content'] = hero.clean(df['content'])

In [8]:
def embedding_by_doc_2_vec(df):
  data = df['content']
  tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
  # df['tagged_data'] = tagged_data
  max_epochs = 5
  vec_size = 20
  alpha = 0.025

  model = Doc2Vec(#size = vec_size,
    vector_size=vec_size,
                  alpha=alpha, 
                  min_alpha=0.00025,
                  min_count=1,
                  dm =1)
    
  model.build_vocab(tagged_data)

  for epoch in range(max_epochs):
      print('iteration {0}'.format(epoch))
      model.train(tagged_data,
                  total_examples=model.corpus_count,
                  #epochs = model.iter
                  epochs=model.epochs)
      # decrease the learning rate
      model.alpha -= 0.0002
      # fix the learning rate, no decay
      model.min_alpha = model.alpha
  print('here')
  print(model.docvecs['1'])
  return model

In [9]:
#model_d2v = embedding_by_doc_2_vec(df)

In [10]:
#model_d2v.save("doc2vec.model")
model_d2v = Doc2Vec.load("doc2vec.model")

In [11]:
def embedd_d2v(model, doc):
    return model.infer_vector(doc.split())

df['embed_title_d2v'] = df['title'].apply(lambda x: embedd_d2v(model_d2v, x))
df['embed_content_d2v'] = df['content'].apply(lambda x: embedd_d2v(model_d2v, x))
df

,title,content,embed_title_d2v,embed_content_d2v
0,House Republicans Fret About Winning Their Hea...,WASHINGTON — Congressional Republicans have...,"[-0.00020618885, 0.0017079205, 0.012472818, -0...","[-0.48268387, -0.037932932, -0.84915787, 5.011..."
1,Rift Between Officers and Residents as Killing...,"After the bullet shells get counted, the blood...","[-0.051277727, 0.3151049, 0.024677651, -0.0277...","[3.0910127, 0.4342424, -0.098031476, 0.1685604..."
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...","When Walt Disney’s “Bambi” opened in 1942, cri...","[-0.04005891, 0.54228365, 0.06120843, -0.02935...","[1.2332672, 2.1408443, 2.617472, -3.2744303, 0..."
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...","Death may be the great equalizer, but it isn’t...","[-0.075641185, 0.1533827, 0.02122116, 0.104032...","[-1.9783455, 2.961961, 6.096002, -0.82950646, ..."
4,Kim Jong-un Says North Korea Is Preparing to T...,"SEOUL, South Korea — North Korea’s leader, ...","[-0.071448535, 0.03868655, 0.010632665, -0.010...","[0.69137114, 3.069012, 3.005353, 2.6914027, 5...."
...,...,...,...,...
7798,U.N. Relief Official Calls Crisis in Aleppo th...,The top aid official at the United Nations gav...,"[-0.063712195, 0.20381376, 0.03313346, 0.06360...","[1.4032047, 1.8695139, 0.8486812, 1.3467678, 3..."
7799,Federal Judge Curbs Enforcement of North Carol...,A federal judge on Friday curbed the enforceme...,"[0.022196058, 0.022633335, -0.01483866, 0.0132...","[0.24696188, -2.430637, -2.117606, 2.1944127, ..."
7800,Mexicans Accuse President of ‘Historic Error’ ...,MEXICO CITY — If President Enrique Peña Nie...,"[-0.014848642, 0.005586029, -0.007927232, 0.00...","[0.4819322, 2.1651163, 0.16808361, 2.2401412, ..."
7801,"U.S. Presses for Truce in Syria, With Its Larg...","HANGZHOU, China — The image of a Syrian ...","[-0.026488753, 0.35839713, -0.010913456, 0.044...","[0.65575665, 2.396529, 1.9432513, 3.6524916, 4..."


In [28]:
def cosine_similarity(df):
    return 1 - spatial.distance.cosine(df[0], df[1])

In [13]:
df['d2v_similarity'] = df[['embed_title_d2v', 'embed_content_d2v']].apply(cosine_similarity, axis = 1)
df['d2v_similarity'].mean()

0.37211666850418007

In [14]:
embed_USE = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def embed_with_USE(embedder , x):
    return embedder(x.split('.')).numpy().mean(axis = 0)

df['embed_title_USE'] = df['title'].apply(lambda x: embed_with_USE(embed_USE, x))
df['embed_content_USE'] = df['content'].apply(lambda x: embed_with_USE(embed_USE, x))
df['USE_similarity'] = df[['embed_title_USE', 'embed_content_USE']].apply(cosine_similarity, axis = 1)
df['USE_similarity'].mean()


2022-04-08 12:42:30.145300: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


0.4937262279795504

In [15]:
model_BERT = SentenceTransformer(r"sentence-transformers/paraphrase-MiniLM-L6-v2")

def embed_with_USE(model , x):
    return model.encode(x)

df['embed_title_BERT'] = df['title'].apply(lambda x: embed_with_USE(model_BERT, x))
df['embed_content_BERT'] = df['content'].apply(lambda x: embed_with_USE(model_BERT, x))
df['BERT_similarity'] = df[['embed_title_BERT', 'embed_content_BERT']].apply(cosine_similarity, axis = 1)
df['BERT_similarity'].mean()

0.5456832346447217

In [1]:
!pip uninstall tensorflow -y
!pip uninstall tensorflow-cloud -y
!pip install -U tensorflow==1.15
!pip uninstall pytorch-lightning -y
!pip uninstall tensorflow-probability -y

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Successfully uninstalled tensorflow-2.8.0
  Using cached tensorflow-1.15.0-cp37-cp37m-macosx_10_11_x86_64.whl (124.0 MB)
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
  Using cached tensorflow_estimator-1.15.1-py2.py3-none-any.whl (503 kB)
  Using cached tensorboard-1.15.0-py3-none-any.whl (3.8 MB)
Processing /Users/max/Library/Caches/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3/gast-0.2.2-py3-none-any.whl
  Using cached Keras_Applications-1.0.8-py3-none-any.whl (50 kB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
You should consider upgrading via the '/Users/max/.pyenv/versions/3.7.11/bin/python3.7 -m pip 

In [11]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.load("https://tfhub.dev/google/elmo/2")

def create_elmo_embeddings(data):
    return_ls = []
    for dat in data.tolist():
        embed=elmo.signatures['default']( tf.convert_to_tensor([dat]))["elmo"]
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            sess.run(tf.tables_initializer())
            out_x=sess.run(embed)
            #out_y=ses.run(tf.reduce_mean(embed,1))
        return_ls.append(out_x)
    return return_ls

temp1 = create_elmo_embeddings(df['content'][:5])

In [2]:
import tensorflow as tf
import tensorflow_hub as tf_hub
elmo = tf_hub.Module("https://tfhub.dev/google/elmo/2")
embeddings = elmo(
    ["the cat is on the mat", "dogs are in the fog"],
    signature="default",
    as_dict=True)["elmo"]
embeddings

/Users/max/.pyenv/versions/3.7.11/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


<tf.Tensor 'module_apply_default/aggregation/mul_3:0' shape=(2, 6, 1024) dtype=float32>

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

max_feat = 10000

tfidf = TfidfVectorizer(min_df=5, max_features=max_feat, ngram_range=(1,3),
                        strip_accents='unicode',
                        lowercase =True, analyzer='word', token_pattern=r'\w+',
                        use_idf=True, smooth_idf=True, sublinear_tf=True, 
                        stop_words = 'english')
svdT = TruncatedSVD(n_components=390)
tfidf.fit(df['content'])
df['embed_content_tfidf'] = svdT.fit_transform(tfidf.transform(df['content'])).tolist()
df['embed_title_tfidf'] = svdT.fit_transform(tfidf.transform(df['title'])).tolist()
df['tfidf_similarity'] = df[['embed_title_tfidf', 'embed_content_tfidf']].apply(cosine_similarity, axis = 1)
df['tfidf_similarity'].mean()


/Users/max/.pyenv/versions/3.7.11/lib/python3.7/site-packages/scipy/spatial/distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


0.31220945733240824

In [19]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

import os
root_folder='.'
data_folder_name='data'

DATA_PATH = os.path.abspath(os.path.join(root_folder, data_folder_name))

glove_filename='glove.6B.100d.txt'
#glove_path = os.path.abspath(os.path.join(DATA_PATH, glove_filename))
#glove_input_file = glove_filename
#word2vec_output_file = glove_filename+'.word2vec'
#glove2word2vec(glove_path, word2vec_output_file)

# load the Stanford GloVe model
word2vec_output_file = glove_filename+'.word2vec'
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [30]:
import numpy as np
def embed_glove(s):
    return_ls = []
    for sentence in s:
        array = []
        for word in sentence.split():
            try:
                array.append(model.get_vector(word))
            except:
                pass
        array = np.array(array)
        #print(array.shape)
        return_ls.append(array.mean(axis = 0))

    return return_ls
#Show a word embedding
df['embed_content_glove'] = embed_glove(df['content'])
df['embed_title_glove'] = embed_glove(df['title'])
df['glove_similarity'] = df[['embed_title_glove', 'embed_content_glove']].apply(cosine_similarity, axis = 1)
df['glove_similarity'].mean()

/Users/max/.pyenv/versions/3.7.11/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: Mean of empty slice.
  del sys.path[0]


0.8266875235549795

In [34]:
from gensim.models import FastText

sentences = df['content']
sentences = [s.split() for s in sentences]
model = FastText(sentences)

In [ ]:
def embed_fasttext(s):
    return_ls = []
    for sentence in s:
        array = []
        for word in sentence.split():
            try:
                array.append(model.wv[word])
            except:
                pass
        array = np.array(array)
        #print(array.shape)
        return_ls.append(array.mean(axis = 0))

    return return_ls

df['embed_content_fasttext'] = embed_fasttext(df['content'])
df['embed_title_fasttext'] = embed_fasttext(df['title'])
df['fasttext_similarity'] = df[['embed_title_fasttext', 'embed_content_fasttext']].apply(cosine_similarity, axis = 1)
df['fasttext_similarity'].mean()